In [1]:
# 检查torch的安装以及gpu的使用
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# 检查MMAction2的安装
import mmaction
print(mmaction.__version__)

# 检查mmcv的安装
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu113 True
0.15.0
11.3
MSVC 192930137


In [2]:
from mmaction.apis import inference_recognizer, init_recognizer

# 选择tsn对应的配置文件
config = 'configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py'
# 加载上面下载的checkpoint文件
checkpoint =  'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# 初始化模型
print(config)
model = init_recognizer(config, checkpoint, device='cuda:1')

configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py


C:\Users\15281\anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load checkpoint from local path: checkpoints/swin_base_patch244_window877_kinetics400_1k.pth


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random

# Function to read and randomly split a txt file
def read_and_split_txt(file_path, test_size=0.2):
    # Read the txt file into a pandas DataFrame
    data = pd.read_csv(file_path, delimiter=",", header=None)
    
    # Randomly shuffle the DataFrame
    data = data.sample(frac=1).reset_index(drop=True)
    
    # Split the data into features and target variable
    X = data  # Assuming the last column is the target variable
    y = data.iloc[:, -1]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    return X_train, X_test, y_train, y_test

# Example usage
file_path = './video_data/硬样本_A_分类.csv'
X_train, X_test, y_train, y_test = read_and_split_txt(file_path)
X_train.to_csv('./video_data/cu/train2.txt', index=False, header=False, sep=' ')
X_test.to_csv('./video_data/cu/val_file2.txt', index=False, header=False, sep=' ')
X_train.to_csv('./video_data/train2.txt', index=False, header=False, sep=' ')
X_test.to_csv('./video_data/val_file2.txt', index=False, header=False, sep=' ')


In [4]:
from mmcv.runner import set_random_seed
# 获得tsn对应的配置文件cfg
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py')
# 修改数据集类型和各个文件路径
cfg.dataset_type = 'VideoDataset'
cfg.data_root = './video_data/train_video/'
cfg.data_root_val = './video_data/val_video/'
cfg.ann_file_train = './video_data/train2.txt'
cfg.ann_file_val = './video_data/val_file2.txt'
cfg.ann_file_test = './video_data/val_file2.txt'


cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = './video_data/val_file2.txt'
cfg.data.test.data_prefix = './video_data/val_video/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = './video_data/train2.txt'
cfg.data.train.data_prefix = './video_data/train_video/'
cfg.data.train.data_prefix = './video_data/train_video/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = './video_data/val_file2.txt'
cfg.data.val.data_prefix = './video_data/val_video/'

# 这里用于确认是否使用到omnisource训练
cfg.setdefault('omnisource', False)
# 修改cls_head中类别数为3
cfg.model.cls_head.num_classes = 3
# 使用预训练好的tsn模型
cfg.load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# 设置工作目录
cfg.work_dir = './tutorial_exps1'

# 由于是单卡训练，修改对应的lr
#cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
#设置epoches
cfg.total_epochs = 20
# 设置存档点间隔减少存储空间的消耗
cfg.checkpoint_config.interval = 10
# 设置日志打印间隔减少打印时间
cfg.log_config.interval = 5

# 固定随机种子使得结果可复现
cfg.seed = 40
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1,2)

# 打印所有的配置参数
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='Recognizer3D',
    backbone=dict(
        type='SwinTransformer3D',
        patch_size=(2, 4, 4),
        embed_dim=128,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        window_size=(8, 7, 7),
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        patch_norm=True),
    cls_head=dict(
        type='I3DHead',
        in_channels=1024,
        num_classes=3,
        spatial_type='avg',
        dropout_ratio=0.5),
    test_cfg=dict(average_clips='prob', max_testing_views=4))
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = './video_data/train_video/'
data_root_val = './v

In [5]:
import os.path as osp
import os
def clean():
    # Get a list of all files in the current directory
    files_in_directory = os.listdir()
    filtered_files = [file for file in files_in_directory if file.endswith(".pt")]
    # Loop through the list and remove each file
    for file in filtered_files:
        os.remove(file)
def cleancsv(directory):
    # 指定要清除CSV文件的目录
    # 遍历目录中的所有文件
    for filename in os.listdir(directory):
        # 检查文件扩展名是否为'.csv'
        if filename.endswith('.csv'):
            # 构造完整的文件路径
            file_path = os.path.join(directory, filename)
            # 删除文件
            os.remove(file_path)

In [6]:
#按排序重建训练文件
import pandas as pd
import numpy as np
import cv2
data = pd.read_csv("./video_data/硬样本——A.csv",sep=",", engine='python', encoding='utf-8')
data2 = pd.read_csv("./video_data/cu/train2.txt", delimiter=" ", header=None)
def select(data,data2,lr):
    ko = []
    for i in range(0,len(data2[0])):
        for j in range(0,len(data["名称"])):
            if(data2[0][i] == data["名称"][j]):
                if(data["dis"][j]<lr):
                    #print(i)
                    ko.append(i)
    data3 = data2.loc[ko, :]
    return data3
#data4 = select(data,data2,2.5)

In [7]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model,train_model_1

import mmcv
import mmcv
import os
import glob

# 构建动作识别模型
# 创建工作目录并训练模型
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
#datasets = [build_dataset(cfg.data.train)]
# 构建数据集
lr = 8.5#8.5
path = "./save2/"
#train_model(model, datasets, cfg, distributed=False, validate= True)
for i in range(0,20):
    clean()
    cleancsv(path)
    lr = lr+1#1
    data4 = select(data,data2,lr)
    data4.to_csv('./video_data/train2.txt', index=False, header=False, sep=' ')
    datasets = [build_dataset(cfg.data.train)]
    #model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
    cfg.lr_config = dict(policy='step', step=[40, 80])
    train_model(model, datasets, cfg, distributed=False, validate= True)
    #del model
    #torch.cuda.empty_cache()
    #del runner
    cfg.load_from = './tutorial_exps/epoch_20.pth'

C:\Users\15281\anaconda3\lib\site-packages\apex-0.1-py3.8.egg\apex\__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


2024-06-28 21:27:57,661 - mmaction - INFO - load checkpoint from local path: checkpoints/swin_base_patch244_window877_kinetics400_1k.pth
2024-06-28 21:27:57,949 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 1024]) from checkpoint, the shape in current model is torch.Size([3, 1024]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([3]).
2024-06-28 21:27:57,960 - mmaction - INFO - Start running, host: 15281@实验室, work_dir: H:\code\视频组代码\Video_Swin_Transformer\tutorial_exps1
2024-06-28 21:27:57,961 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) DistOptimizerHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 9/9, 0.6 task/s, elapsed: 16s, ETA:     0s

2024-06-28 21:29:12,045 - mmaction - INFO - Evaluating top_k_accuracy ...
2024-06-28 21:29:12,045 - mmaction - INFO - 
top1_acc	0.4444
top5_acc	1.0000
2024-06-28 21:29:12,046 - mmaction - INFO - Evaluating mean_class_accuracy ...
2024-06-28 21:29:12,047 - mmaction - INFO - 
mean_acc	0.5667


...

mean_class_accuracy
[2 0 2 2 1 0 1 2 2]
[2, 0, 2, 2, 1, 1, 0, 2, 2]


2024-06-28 23:42:57,367 - mmaction - INFO - Epoch [16][5/17]	lr: 7.813e-07, eta: 0:01:09, time: 1.819, data_time: 1.368, memory: 9375, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.8022, loss: 0.8022
2024-06-28 23:43:00,303 - mmaction - INFO - Epoch [16][10/17]	lr: 7.813e-07, eta: 0:01:05, time: 0.587, data_time: 0.202, memory: 9375, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.7366, loss: 0.7366
2024-06-28 23:43:02,168 - mmaction - INFO - Epoch [16][15/17]	lr: 7.813e-07, eta: 0:01:00, time: 0.373, data_time: 0.002, memory: 9375, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls: 0.7526, loss: 0.7526
2024-06-28 23:43:13,571 - mmaction - INFO - Epoch [17][5/17]	lr: 7.813e-07, eta: 0:00:54, time: 1.720, data_time: 1.284, memory: 9375, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.7415, loss: 0.7415
2024-06-28 23:43:16,172 - mmaction - INFO - Epoch [17][10/17]	lr: 7.813e-07, eta: 0:00:50, time: 0.520, data_time: 0.140, memory: 9375, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls: 0.8066, 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 9/9, 0.4 task/s, elapsed: 21s, ETA:     0s

2024-06-28 23:44:36,056 - mmaction - INFO - Evaluating top_k_accuracy ...
2024-06-28 23:44:36,057 - mmaction - INFO - 
top1_acc	0.8889
top5_acc	1.0000
2024-06-28 23:44:36,057 - mmaction - INFO - Evaluating mean_class_accuracy ...
2024-06-28 23:44:36,058 - mmaction - INFO - 
mean_acc	0.8333
2024-06-28 23:44:36,058 - mmaction - INFO - Epoch(val) [20][9]	top1_acc: 0.8889, top5_acc: 1.0000, mean_class_accuracy: 0.8333


mean_class_accuracy
[2 0 2 2 1 0 0 2 2]
[2, 0, 2, 2, 1, 1, 0, 2, 2]


In [8]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
cfg.data.val.type = 'VideoDataset'
# 构建测试数据集
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[1])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8/8, 0.3 task/s, elapsed: 24s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.8750
top5_acc	1.0000

Evaluating mean_class_accuracy ...
mean_class_accuracy
[1 1 1 1 2 0 2 2]
[1, 0, 1, 1, 2, 0, 2, 2]

mean_acc	0.8333
top1_acc: 0.8750
top5_acc: 1.0000
mean_class_accuracy: 0.8333


In [7]:
#打分矩阵
from sklearn.metrics import classification_report
import numpy as np
a = [1, 0, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 0, 2, 0, 2, 1, 1, 1, 1, 1, 2, 2]
b = [0, 0, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 0, 0, 0, 2, 1, 1, 1, 1, 1, 2, 1]
print(classification_report(a,b))

              precision    recall  f1-score   support

           0       0.71      1.00      0.83         5
           1       0.80      0.86      0.83        14
           2       0.93      0.76      0.84        17

    accuracy                           0.83        36
   macro avg       0.81      0.87      0.83        36
weighted avg       0.85      0.83      0.83        36



In [22]:
torch.save(model, 'Swin-SPL-noiseB1.pth')

In [7]:
# 打开两个文件进行读取
with open('./video_data/cu2/train.txt', 'r') as file1, open('./video_data/cu/train2.txt', 'r') as file2:
    # 读取文件内容
    content1 = file1.read()
    content2 = file2.read()

# 将两个文件的内容拼接
combined_content = content1 + content2

# 将拼接后的内容写入到新的文件中
with open('./video_data/train_all.txt', 'w') as outfile:
    outfile.write(combined_content)

In [14]:
cfg.ann_file_val = './video_data/train_all.txt'
cfg.ann_file_test = './video_data/train_all.txt'
cfg.data.test.ann_file = 'train_all.txt'
cfg.data.test.data_prefix = './video_data/val_video/'
cfg.data.val.ann_file = './video_data/train_all.txt'
cfg.data.val.data_prefix = './video_data/val_video/'

In [15]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
cfg.data.val.type = 'VideoDataset'
# 构建测试数据集
#model = torch.load('Swin-SPL-noiseB.pth')
path = "./save2/"
cleancsv(path)
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[1])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
#eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 177/177, 0.6 task/s, elapsed: 319s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.7910
top5_acc	1.0000

Evaluating mean_class_accuracy ...
mean_class_accuracy
[2 2 2 1 1 2 0 2 1 2 2 1 2 1 1 2 2 1 0 1 2 2 1 2 2 2 0 1 0 0 1 2 0 2 2 2 2
 2 1 2 2 2 1 0 1 1 0 0 0 2 2 2 2 2 2 2 2 1 0 0 0 2 1 1 2 1 2 2 2 2 0 1 1 2
 2 0 2 0 2 0 1 2 2 2 1 2 2 2 2 2 0 2 2 2 0 2 1 2 2 0 0 2 0 0 1 2 1 2 0 0 1
 2 0 1 1 0 2 1 2 2 2 2 0 2 1 2 2 0 2 2 2 1 2 2 2 0 1 2 2 2 0 2 0 2 0 0 0 1
 0 0 2 2 1 2 0 2 2 0 2 0 2 0 1 1 0 0 1 0 2 0 0 2 2 2 0 2 2]
[2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 2, 2, 0, 1, 0, 0, 1, 2, 0, 2, 2, 2, 0, 2, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 2, 2, 1, 2, 2, 0, 2, 1, 1, 0, 0, 0, 2, 1, 1, 2, 1, 1, 2, 2, 1, 0, 1, 1, 2, 2, 0, 0, 0, 1, 0, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 1, 0, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 2, 0, 2, 0, 2, 1, 0, 2, 2, 2, 1, 2, 2, 0, 0, 1, 2, 0, 2,

In [10]:
import os
import shutil

# 指定源目录和目标目录
source_dir = './save2/'
target_dir = './save2/alltrain-n/'
cleancsv(target_dir)
# 遍历源目录
for filename in os.listdir(source_dir):
    # 检查文件扩展名是否为.csv
    if filename.endswith('.csv'):
        # 构建完整的文件路径
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)

        # 复制文件
        shutil.copy(src_path, dst_path)
        print(f'Copied {filename} to {target_dir}')

Copied 13_40_50.csv to ./save2/alltrain-n/
Copied 13_40_52.csv to ./save2/alltrain-n/
Copied 13_40_54.csv to ./save2/alltrain-n/
Copied 13_40_56.csv to ./save2/alltrain-n/
Copied 13_40_58.csv to ./save2/alltrain-n/
Copied 13_40_59.csv to ./save2/alltrain-n/
Copied 13_41_1.csv to ./save2/alltrain-n/
Copied 13_41_10.csv to ./save2/alltrain-n/
Copied 13_41_11.csv to ./save2/alltrain-n/
Copied 13_41_13.csv to ./save2/alltrain-n/
Copied 13_41_15.csv to ./save2/alltrain-n/
Copied 13_41_16.csv to ./save2/alltrain-n/
Copied 13_41_18.csv to ./save2/alltrain-n/
Copied 13_41_20.csv to ./save2/alltrain-n/
Copied 13_41_22.csv to ./save2/alltrain-n/
Copied 13_41_23.csv to ./save2/alltrain-n/
Copied 13_41_25.csv to ./save2/alltrain-n/
Copied 13_41_27.csv to ./save2/alltrain-n/
Copied 13_41_28.csv to ./save2/alltrain-n/
Copied 13_41_3.csv to ./save2/alltrain-n/
Copied 13_41_30.csv to ./save2/alltrain-n/
Copied 13_41_32.csv to ./save2/alltrain-n/
Copied 13_41_33.csv to ./save2/alltrain-n/
Copied 13_41_

In [11]:
# 打开两个文件进行读取
with open('./video_data/cu2/val_file.txt', 'r') as file1, open('./video_data/cu/val_file2.txt', 'r') as file2:
    # 读取文件内容
    content1 = file1.read()
    content2 = file2.read()

# 将两个文件的内容拼接
combined_content = content1 + content2

# 将拼接后的内容写入到新的文件中
with open('./video_data/val_all.txt', 'w') as outfile:
    outfile.write(combined_content)

In [16]:
cfg.ann_file_val = './video_data/val_all.txt'
cfg.ann_file_test = './video_data/val_all.txt'
cfg.data.test.ann_file = 'train_all.txt'
cfg.data.test.data_prefix = './video_data/val_video/'
cfg.data.val.ann_file = './video_data/val_all.txt'
cfg.data.val.data_prefix = './video_data/val_video/'

In [17]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
cfg.data.val.type = 'VideoDataset'
# 构建测试数据集
#model = torch.load('Swin-SPL-noiseB.pth')
path = "./save2/"
cleancsv(path)
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[1])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
#eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 45/45, 0.6 task/s, elapsed: 82s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.7333
top5_acc	1.0000

Evaluating mean_class_accuracy ...
mean_class_accuracy
[0 0 1 1 1 2 1 0 2 2 2 0 2 0 1 1 1 2 2 2 2 2 0 1 2 0 2 2 1 2 2 2 0 0 2 1 2
 0 2 2 1 0 0 2 2]
[0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 1, 0, 0, 2, 1, 2, 0, 2, 2, 1, 1, 0, 2, 2]

mean_acc	0.8032
top1_acc: 0.7333
top5_acc: 1.0000
mean_class_accuracy: 0.8032


In [13]:
import os
import shutil

# 指定源目录和目标目录
source_dir = './save2/'
target_dir = './save2/allval-n/'
cleancsv(target_dir)
# 遍历源目录
for filename in os.listdir(source_dir):
    # 检查文件扩展名是否为.csv
    if filename.endswith('.csv'):
        # 构建完整的文件路径
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)

        # 复制文件
        shutil.copy(src_path, dst_path)
        print(f'Copied {filename} to {target_dir}')

Copied 13_46_11.csv to ./save2/allval-n/
Copied 13_46_13.csv to ./save2/allval-n/
Copied 13_46_14.csv to ./save2/allval-n/
Copied 13_46_16.csv to ./save2/allval-n/
Copied 13_46_18.csv to ./save2/allval-n/
Copied 13_46_19.csv to ./save2/allval-n/
Copied 13_46_21.csv to ./save2/allval-n/
Copied 13_46_23.csv to ./save2/allval-n/
Copied 13_46_25.csv to ./save2/allval-n/
Copied 13_46_26.csv to ./save2/allval-n/
Copied 13_46_28.csv to ./save2/allval-n/
Copied 13_46_30.csv to ./save2/allval-n/
Copied 13_46_31.csv to ./save2/allval-n/
Copied 13_46_33.csv to ./save2/allval-n/
Copied 13_46_35.csv to ./save2/allval-n/
Copied 13_46_37.csv to ./save2/allval-n/
Copied 13_46_38.csv to ./save2/allval-n/
Copied 13_46_4.csv to ./save2/allval-n/
Copied 13_46_40.csv to ./save2/allval-n/
Copied 13_46_42.csv to ./save2/allval-n/
Copied 13_46_44.csv to ./save2/allval-n/
Copied 13_46_45.csv to ./save2/allval-n/
Copied 13_46_47.csv to ./save2/allval-n/
Copied 13_46_49.csv to ./save2/allval-n/
Copied 13_46_50.c

In [ ]:
torch.save(model,'total_model.pth')